In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import metrics
import xgboost as xgb

import nltk
from nltk import word_tokenize

import string, six
from sklearn.feature_extraction.text import CountVectorizer
import re
rc = re.compile

which = lambda lst:list(np.where(lst)[0])

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
pd.set_option('display.max_colwidth', -1)
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.tsv', sep= "\t")
test = pd.read_csv('../input/test.tsv', sep= "\t")
sub = pd.read_csv('../input/sampleSubmission.csv', sep= ",")

In [ ]:
print('SHAPE TRAIN:', train.shape) 
train.head()

In [ ]:
print('SHAPE TEST:',test.shape) 
test.head()

In [ ]:
fig, ax = plt.subplots(1, 1,dpi=100)
sentiment_labels = train.Sentiment.value_counts().index
sentiment_count = train.Sentiment.value_counts()
sns.barplot(x=sentiment_labels,y=sentiment_count)
ax.set_ylabel('Count')    
ax.set_xlabel('Sentiment Label')
ax.set_xticklabels(sentiment_labels , rotation=30 );

In [ ]:
def clean_text(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('[%s]' % re.escape(string.digits), '', text)
    text = re.sub('[%s]' % re.escape(' +'), ' ', text)
    text = text.lower()
    text = text.strip()
    return text

In [ ]:
train['Phrase'] = train['Phrase'].apply(lambda x: clean_text(x))
test['Phrase'] = test['Phrase'].apply(lambda x: clean_text(x))

In [ ]:
def tdm_df(df, col, prefix = u"tf_", xColNames = None, **kwargs):
    #initialize the  vectorizer
    vectorizer = CountVectorizer(**kwargs)
    x1 = vectorizer.fit_transform(df[col])
    return({
        'dfm':x1.tocsr(),
        'dfmBase' : [prefix + s for s in vectorizer.get_feature_names()]
    })

In [ ]:
title_dfm = tdm_df(
    df=train.append(test), 
    col='Phrase', 
    prefix='', 
    analyzer = 'word', 
    ngram_range = (1,2), 
#     stop_words = stopwords.words('english'), 
    lowercase = True,
    min_df=15,
    binary = True,
    tokenizer = word_tokenize
)
title_dfm_base = title_dfm['dfmBase']
title_dfm = title_dfm['dfm']

In [ ]:
title_dfm_base_train = title_dfm_base[:len(train)]
title_dfm_base_test = title_dfm_base[len(train):]
title_dfm_train = title_dfm[:len(train)]
title_dfm_test = title_dfm[len(train):]

In [ ]:
X = title_dfm_train
y = train['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
gbm = xgb.XGBClassifier(max_depth=25, 
                        learning_rate=0.1, 
                        n_estimators=300, 
                        objective='multi:softprob', 
                        booster='gbtree', 
                        n_jobs=1, 
                        nthread=20).fit(X_train, y_train,verbose=True)

In [ ]:
predictions = gbm.predict(X_test)
gbm22 = accuracy_score(y_test, predictions)

In [ ]:
xgb_metric = metrics.classification_report(y_test, predictions)
print('XGBClassifier')
print('Accuracy: {:.4%}'.format(gbm22))
print('Matrix metrics:') 
print(xgb_metric)

In [ ]:
sub['Sentiment'] =  gbm.predict(title_dfm_test)
sub.to_csv("xgb_test.csv", index=False)